# Setting up the regional configs at 1/36° from NACHOS12.L75 with NEMO 4.0 rev 10650

## What region

We want a geographical area that will run at 1/24° at the same cost than a full NACHOS run at 1/12° : 40° x 25° or 20° x 50° for each.

We want to cover LatMix and OSMOSIS campaign, so a Gulf Stream config and a  European coast config.

First guess :
 - GS : 82°W-35°W x 28°N-50°N
 - EU : 24.25°W-5°E x 32°N-65.5°N
 
Update on June 24th :
 - GS : 82°W-40°W x 27°N-48°N
 - EU : Iceland to Norway to Madeira Island


### The grid -> done

on cal1, at /mnt/meom/workdir/albert/GS36.L75/GS36.L75-I :

 - extract the region from eNATL36 coordinates file : in 
     - ```ulimit -s unlimited; cdffindij -w -82 -40 27 48 -c /mnt/meom/MODEL_SET/eNATL36X/eNATL36X-I/eNATL36X_coordinates_time_v3.0.nc``` 
     - j'enlève 57 points en x pour ne pas avoir le bout du golfe du mexique (vu dans le mesh_mask)
     - ```ncks -O -F -d x,632,2099 -d y,1406,2355  /mnt/meom/MODEL_SET/eNATL36X/eNATL36X-I/eNATL36X_coordinates_time_v3.0.nc GS36_coordinates.nc```
 
### Bathymetry -> done

on cal1, at /mnt/meom/workdir/albert/GS36.L75/GS36.L75-I :

  - Jean-Marc did all the work already, bathy eNATL36 is on cal1 : /mnt/meom/MODEL_SET/eNATL36X/eNATL36X-I/eNATL36X_bathy_gebco2014_v2.3.nc
  - check sea mounts in GS -> done by Jean-Marc
  - cut-off GS36 : ```ncks -O -F -d x,632,2099 -d y,1406,2355 /mnt/meom/MODEL_SET/eNATL36X/eNATL36X-I/eNATL36X_bathy_gebco2014_v2.3.nc GS36_bathy_from_eNATL36_v2.3.nc``` 
 
### Domain -> done

on occigen, at /scratch/cnt0024/hmg2840/albert7a/GS36.L75/GS36.L75-I :

  - use the DOMAIN_cfg tool to construct the domain from grid and bathymetry files :
    - copy the arch file for occigen : ```cp /home/albert7a/CONFIGS/CONFIG_GS24.L75/GS24.L75-MAA008/arch/arch-X64_OCCIGEN2.fcm /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF/NEMO4/arch/.```
    - in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF/NEMO4/tools
         - ```./maketools -n DOMAINcfg -m X64_OCCIGEN2```
         - copy the directory into /scratch/cnt0024/hmg2840/albert7a/GS36.L75/GS36.L75-I/DOMAINcfg
         - make the change in namelist_cfg corresponding to the config (vertical grid)
         - link the coordinates and bathymetry into coordinates.nc bathy_meter.nc
         - ```ulimit -s unlimited; ./make_domain_cfg.exe```
         - file GS36.L75_domain_cfg.nc

### Run-offs -> done

on cal1, at /mnt/meom/workdir/albert/GS36.L75/GS36.L75-I :

  - Jean-Marc did all the work already, /mnt/meom/MODEL_SET/eNATL36X/eNATL36X-I/eNATL36X_runoff_v2.2.2.nc
  - ```ncks -O -F -d x,632,2099 -d y,1406,2355  /mnt/meom/MODEL_SET/eNATL36X/eNATL36X-I/eNATL36X_runoff_v3.3.2.nc GS36_runoff.nc```
  
### The code -> done

  - download latest version of DCM : ``` git clone https://github.com/meom-group/DCM.git DCM_4.0-10650```
  - in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF, run getnemoref.sh 
  - add file 4.0-10650 in /home/albert7a/modules/NEMODRAK and add ```module load NEMODRAK/4.0-10650``` in .bashrc
  - get the modifications I added in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/RUNTOOLS/lib 
 
  - reference configuration /home/albert7a/CONFIGS/CONFIG_GS24.L75/GS24.L75-MAA008

### Compilation -> done
    
    - mkconfdir_local GS36.L75 MAA001
    - in /home/albert7a/CONFIGS/CONFIG_GS36.L75/GS36.L75-MAA001
        - copy makefile, CPP.keys, arch/arch-X64_OCCIGEN2.fcm from /home/albert7a/CONFIGS/CONFIG_GS24.L75/GS24.L75-MAA008
        - j'enlève le modèle de glace in CPP.keys et makefile
        - make copyconfig, install 

### Mesh_mask -> done

on occigen, cdftool cdfmeshmask -> segmentation fault à la création du fichier
    - run GS36.L75-MAA001 à vide avec fichier GS36.L75_domain_cfg.nc
    - in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF/NEMO4/tools
        - ./maketools -n REBUILD_NEMO -m X64_OCCIGEN2
    - cp /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF/NEMO4/tools/REBUILD_NEMO/rebuild_nemo* /scratch/cnt0024/hmg2840/albert7a/TMPDIR_GS36.L75-MAA001
    - ./rebuild_nemo mesh_mask 1152
    - cp mesh_mask.nc /scratch/cnt0024/hmg2840/albert7a/GS36.L75/GS36.L75-I/GS36.L75-MAA001_mesh_mask.nc
  
### Initial conditions and damping files -> done
  - sosie on GLORYS2V4 :
      - in /scratch/cnt0024/hmg2840/albert7a/DEV/MK_INI_GLORYS2GS36
          - from MK_INI_GLORYS2GS24 copy :
              - make_sosie.ksh, tmp_namelist, tmp_namelist2D 
              - make_init.ksh
          - link GS36.L75_mesh_mask.nc 
          - modify tmp_namelist and tmp_namelist2D
          - ```./make_sosie.ksh```
          - ```./make_init.ksh``` 
      
### BDY -> done

  - sosie on NACHOS12-MAA4001 (2010) and MAA4002 (2011-2013) outputs :
      - in /scratch/cnt0024/hmg2840/albert7a/DEV/MK_BDY_NACHOS12GS36
          - from MK_BDY_GS24-sosie-direct copy :
              - make_bdy_target_from_mask_GS24.ksh
              - make_sosie_NACHOS12-BDYS.ksh, tmp_namelist2D, tmp_namelist
              
          - ```./make_bdy_target_from_mask_GS36.ksh``` to get the slices of mesh_mask corresponding to the boundaries
          - modify tmp_namelist and tmp_namelist2D
          - ```./make_sosie_NACHOS12-BDYS_monthly_job.ksh```
          - ```./make_concat_flip.ksh``` : concat the rims and flip dimensions when needed
          - put NACHOS12-GS36.L75_N/S/EbdyT/S/U/V/ssh_y2010.nc in GS36.L75-I/BDY
      - in  /scratch/cnt0024/hmg2840/albert7a/DEV/
          - ```git clone https://github.com/meom-group/DCM.git DCM_16072019```
      - in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_16072019/DCMTOOLS/DRAKKAR/NEMO4/tools/BDY_TOOLS/src
          - ```make bdy_coord_create```
          - ```cp -r BDY_TOOLS $WORKDIR/DEV/. ```
          - modify namelist_GS36.L75 for 3 boundaries and rim of 3
          - ```src/bdy_coord_create -n namelist_GS36.L75 ```
          - put sbdy_coordinate_file.nc, sbdy_coordinate_file.nc and sbdy_coordinate_file.nc in GS36.L75-I/BDY  


### Weights for atm forcing -> done

  - in /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF/NEMO4/tools
    - ./maketools -n WEIGHTS -m X64_OCCIGEN2
    - cp -r /scratch/cnt0024/hmg2840/albert7a/DEV/DCM_4.0-10650/DCMTOOLS/NEMOREF/NEMO4/tools/WEIGHTS /scratch/cnt0024/hmg2840/albert7a/DEV/WEIGHTS_4.0-10650
  - in /scratch/cnt0024/hmg2840/albert7a/DEV/WEIGHTS_4.0-10650
      - copy make_weights_dfs5_zgs24.ksh from WEIGHTS
      - copy namelist.skel
      - modify make_weights_DSF5.2-GS36.L75.ksh to mimic mkweight.ksh from WEIGHT
  - build new weights fitted to DFS5.2 :
      - in /scratch/cnt0024/hmg2840/albert7a/DEV/WEIGHTS_4.0-10650
          - ```./make_weights_DSF5.2-GS36.L75.ksh```
          - rename wght_bilinear_.nc, wght_bicubic_.nc and copy to /scratch/cnt0024/hmg2840/albert7a/GS36.L75/GS36.L75-I

### Other files

    - in /scratch/cnt0024/hmg2840/albert7a/DEV/MK_SOSIE_NACHOS12-GS36 sosie over chlaseawifs_c1m-99-05_smooth_NACHOS12 : chlaseawifs_c1m-99-05_smooth_GS36

  
    
### Running
    
    - in /home/albert7a/RUNS
        - copy_files_for_run_nemo40.ksh 
    - in /home/albert7a/RUNS/RUN_GS36.L75/GS36.L75-MAA001
        - get rid of ice namelist and xml
        - in namelist.GS36.L75-MAA001, modify :
            - nn_ice      =  0
            - rn_rdt      =  120.
            - cn_domcfg = "GS36.L75_domain_cfg"
            - sn_tem_ini  = 'votemper_GLORYS025-GS36.L75_y2010' clim=true
            - sn_sal_ini  = 'vosaline_GLORYS025-GS36.L75_y2010' clim=true
            - sn_tem_dmp  = 'votemper_GLORYS025-GS36.L75' clim=false
            - sn_sal_dmp  = 'vosaline_GLORYS025-GS36.L75' clim=false
            - sn_wndi = ... 'wght_bicubic_DSF5-GS36.nc' et 'wght_bilinear_DSF5-GS36.nc'
            - sn_chl      = 'chlaseawifs_c1m-99-05_smooth_GS36'
            - sn_sss      =    'sosaline_GLORYS025-GS36.L75'
            - sn_rnf      = 'GS36_runoff' et sn_cnf      = 'GS36_runoff'
            - ln_bdy
            - cn_resto = 'GS36.L75_resto.nc' => no resto
            - rn_Ud = 0.024 (from ZKERG36 value 33 with JM formula w MAX(e1,e2)=2754
            
    
### Scalability

    - first test 5 days with 1152 nodes for nemo and 24 for xios : 